In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784',version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [3]:
x, y = mnist["data"], mnist["target"]
y=y.astype(np.uint8)
print(x.shape,y.shape)
x_train,x_test,y_train,y_test=x[:42000],x[42000:],y[:42000],y[42000:]

(70000, 784) (70000,)


In [4]:
BATCH_SIZE=250
torch_x_train=torch.from_numpy(x_train).type(torch.LongTensor)
torch_y_train=torch.from_numpy(y_train).type(torch.LongTensor)
torch_x_test=torch.from_numpy(x_test).type(torch.LongTensor)
torch_y_test=torch.from_numpy(y_test).type(torch.LongTensor)
torch_x_train = torch_x_train.view(-1, 1,28,28).float()
torch_x_test = torch_x_test.view(-1,1,28,28).float()
print(torch_x_train.shape)
print(torch_x_test.shape)

torch.Size([42000, 1, 28, 28])
torch.Size([28000, 1, 28, 28])


In [0]:
train=torch.utils.data.TensorDataset(torch_x_train,torch_y_train)
test=torch.utils.data.TensorDataset(torch_x_test,torch_y_test)
train_loader=torch.utils.data.DataLoader(train,batch_size=BATCH_SIZE,shuffle=False)
test_loader=(torch.utils.data.DataLoader(test,batch_size=BATCH_SIZE,shuffle=False))

In [0]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 6
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            var_x_batch = Variable(x_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_x_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch+1, batch_idx*len(x_batch), len(train_loader.dataset), 100*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [0]:
def evaluate(model):
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

In [8]:
class cnn(nn.Module):
  def __init__(self):
    super(cnn, self).__init__()
    self.conv1=nn.Conv2d(1,40,5)
    self.conv2=nn.Conv2d(40,50,3,padding=1)
    self.conv3=nn.Conv2d(50,60,3,padding=1)
    self.fc1=nn.Linear(60*3*3,393)
    self.fc2=nn.Linear(393,159)
    self.fc3=nn.Linear(159,10)
  
  
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, 2, 2)    
    x = x.view(-1,3*3*60)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x, dim=1)
CNN=cnn()
print(CNN)
it = iter(train_loader)
x_batch, y_batch = next(it)

cnn(
  (conv1): Conv2d(1, 40, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(40, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(50, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=540, out_features=393, bias=True)
  (fc2): Linear(in_features=393, out_features=159, bias=True)
  (fc3): Linear(in_features=159, out_features=10, bias=True)
)


In [9]:
CNN.forward(x_batch).size()

torch.Size([250, 10])

In [10]:
fit(CNN,train_loader)

Epoch : 1 [0/42000 (0%)]	Loss: 4.675600	 Accuracy:9.200%
Epoch : 1 [12500/42000 (30%)]	Loss: 0.246928	 Accuracy:76.753%
Epoch : 1 [25000/42000 (60%)]	Loss: 0.065686	 Accuracy:85.988%
Epoch : 1 [37500/42000 (89%)]	Loss: 0.083468	 Accuracy:89.534%
Epoch : 2 [0/42000 (0%)]	Loss: 0.120754	 Accuracy:97.200%
Epoch : 2 [12500/42000 (30%)]	Loss: 0.105498	 Accuracy:97.631%
Epoch : 2 [25000/42000 (60%)]	Loss: 0.032269	 Accuracy:97.949%
Epoch : 2 [37500/42000 (89%)]	Loss: 0.052831	 Accuracy:98.064%
Epoch : 3 [0/42000 (0%)]	Loss: 0.062244	 Accuracy:98.000%
Epoch : 3 [12500/42000 (30%)]	Loss: 0.096505	 Accuracy:98.494%
Epoch : 3 [25000/42000 (60%)]	Loss: 0.025102	 Accuracy:98.594%
Epoch : 3 [37500/42000 (89%)]	Loss: 0.030352	 Accuracy:98.612%
Epoch : 4 [0/42000 (0%)]	Loss: 0.048079	 Accuracy:99.200%
Epoch : 4 [12500/42000 (30%)]	Loss: 0.051913	 Accuracy:98.957%
Epoch : 4 [25000/42000 (60%)]	Loss: 0.011455	 Accuracy:99.038%
Epoch : 4 [37500/42000 (89%)]	Loss: 0.031695	 Accuracy:99.020%
Epoch : 5 [0/

In [11]:
evaluate(CNN)

Test accuracy:0.989% 
